# Subspace VQE based on SNAP-displacement

## Prerequisite

Installation cells for Google Colab users.

In [ ]:
!pip install qutip
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 85.8 MB/s eta 0:00:00


Import libaries.

In [ ]:
import numpy as np
import qutip as qt
import scipy.optimize as sciopt

from functools import partial

In [ ]:
import sys

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Basics

In [ ]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [ ]:
def unpack_params_snap_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (nfock + 1)

    # Unpack
    alpha = X[:ndepth].copy()
    d1 = ndepth * nfock
    theta = X[ndepth:ndepth+d1].reshape((ndepth, nfock))

    return alpha, theta


def pack_params_snap_disp(alpha, theta):
    # Initialize
    ndepth = alpha.shape[0]
    nfock = theta.shape[1]
    dim = (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = alpha.copy()
    d1 = ndepth * nfock
    X[ndepth:ndepth+d1] = theta.reshape(-1)

    return X

## Ansatz

In [ ]:
def qproj00():
    return qt.basis(2, 0).proj()


def qproj11():
    return qt.basis(2, 1).proj()


def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)


def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)


def hadamard():
    op = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    return qt.Qobj(op)

In [ ]:
def qubit_rot(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( qt.sigmax() * np.cos(phi) )
    gen += ( qt.sigmay() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return H.expm()

Selective number-dependent arbitray phase (SNAP) and displacement operator ([reference](https://doi.org/10.1103/PhysRevA.92.040303))

\begin{align*}
U (\alpha, \vec{\theta})
&= S (\vec{\theta}) \: D (\alpha),
\\
S (\vec{\theta})
&= \sum_{n = 0}^{L - 1} \: \exp ( i \: \theta_n ) \: |n \rangle \langle n|,
\\
D (\alpha)
&= e^{ \alpha \: ( a^\dagger - a ) }.
\end{align*}

In [ ]:
def snap_disp_op(alpha, thetavec):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # Initialize
    nfock = thetavec.shape[0]

    # SNAP
    S2 = np.exp(1j * thetavec[0]) * qt.basis(nfock, 0).proj()
    for i in range(1, nfock):
        S2 += np.exp(1j * thetavec[i]) * qt.basis(nfock, i).proj()

    # Rotation
    D2 = qt.displace(nfock, alpha)

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\alpha}, \bar{\theta})
= U (\alpha_{N_d}, \vec{\theta}_{N_d}) \cdots
U (\alpha_1, \vec{\theta}_1),
$$

where $\vec{\alpha}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L} $ is a matrix.


In [ ]:
def snap_disp_ansatz(Xvec, nfock):
    """
    SNAP-displacement ansatz.

    Arguments:
    Xvec -- ansatz parameters
    nfock -- Fock cutoff
    """
    # Initialize
    alphavec, thetamat = unpack_params_snap_disp(Xvec, nfock)
    ndepth = thetamat.shape[0]
    uni = snap_disp_op(alphavec[0], thetamat[0, :])

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = snap_disp_op(alphavec[i], thetamat[i, :])
        uni = ( new_uni * uni )

    return uni

## Hamiltonian

### Basics

Extract from file.

In [ ]:
def read_data_file(fname):
    """
    Extract Pauli words and correspoding coefficients.
    """
    with open(fname, 'r') as file:
        contents = file.readlines()

    paulis = []
    coeffs = []
    for line in contents[1:]:
        tmp_pauli, tmp_coeff = line.rstrip('\n').split(',')
        paulis.append(tmp_pauli)
        coeffs.append(tmp_coeff)

    final_coeffs = []
    for tmp_coeff in coeffs:
        new_coeff = tmp_coeff[1:-1].split('+')[0]
        final_coeffs.append(float(("%.17f" % float(new_coeff)).rstrip('0').rstrip('.')))

    return paulis, np.array(final_coeffs)

Exact Hamiltonian using QuTip.

In [ ]:
def build_op_pauli(pstr):
    """
    Build a QuTip version of a Pauli operator.

    Argument:
    pstr -- Pauli operator as a string
    """
    if pstr == 'I':
        op = qt.qeye(2)
    elif pstr == 'X':
        op = qt.sigmax()
    elif pstr == 'Y':
        op = qt.sigmay()
    elif pstr == 'Z':
        op = qt.sigmaz()

    return op


def build_op_pword(pword):
    """
    Build a qubit operator for a Pauli word.

    Argument:
    pword -- Pauli word as string
    """
    op = build_op_pauli(pword[0])
    for j in range(1, len(pword)):
        op = qt.tensor(op, build_op_pauli(pword[j]))

    return op


def build_ham_exact(pwords, coeffs):
    """
    Build a qubit Hamiltonian.

    Arguments:
    pwords -- set of Pauli words as vectors of strings
    coeffs -- Hamiltonian coefficients
    """
    # Check
    if len(coeffs) != len(pwords):
        raise ValueError("Lengths of coeffs and pwords do not match.")

    # Hamiltonian
    ham = ( coeffs[0] * build_op_pword(pwords[0]) )
    for j in range(1, len(pwords)):
        ham += ( coeffs[j] * build_op_pword(pwords[j]) )

    return ham

### Parametrization

Data for $N_d = 16$.

In [ ]:
def get_xr_snap_four_qubits(xind):
    """
    SNAP-displacement circuit for H[j] with j = 0-3.
    """
    # Initialize
    nfock = 16

    # Parameters
    if xind == 1:
        # IHII Loss = 9.024176106107647e-14
        Xvec_nd16 = np.array([ 1.90380697e+00, 1.06351314e+00, 1.21082929e+00, 2.18701172e+00,
  8.47333615e-01, 1.55030875e+00, 8.68847986e-01, 5.34175422e-01,
  1.65631871e-01, 2.40630736e-01, 2.04242171e+00, 1.63231031e+00,
  2.17344991e+00, 2.59794642e+00, 1.02428915e-01, 4.25932349e-01,
  2.37587264e+00, 3.51769709e+00, 3.01099058e+00, 3.09886483e-01,
  1.89021244e+00, 1.75498189e+00, 3.22733916e+00, 2.09008130e+00,
  1.14941518e+00, 1.28336042e+00, 3.44996847e+00, 1.76325696e+00,
 -4.43167726e-01, 3.27115802e-01, 8.07496831e-01, 2.32270541e+00,
  1.32734400e+00, 3.05451706e+00, 1.11079584e-01, 7.55551462e-01,
  1.69576641e+00, 2.77320835e+00, 9.91373910e-01, 3.06289519e+00,
  3.08034042e+00, 2.89269592e+00, 3.85839098e+00, 2.78003147e+00,
  2.59465500e+00, 1.39893833e+00, 1.53490355e+00, 2.34713655e+00,
 -6.67611885e-01, 1.69794870e+00, 3.46327494e-01, 1.66372033e+00,
  5.55053427e-02, 1.23784952e+00, 1.05891366e+00, 3.66365580e-01,
  2.34117477e+00, 1.59859122e+00, 1.67182875e+00, 2.45028332e-01,
  1.58751210e+00, 4.09758986e-01,-3.82560466e-01, 1.69237679e+00,
  3.49137341e+00, 1.67675151e+00, 1.45118876e+00, 2.06203318e+00,
  3.55400457e+00, 6.70945962e-01, 6.16855300e-01,-1.85282342e+00,
  1.65857550e+00, 2.31765157e+00, 5.75361941e-02, 3.13397545e+00,
  2.72846154e+00, 1.40350908e+00, 1.89648641e+00, 9.20752759e-02,
  2.84308428e+00, 2.70177571e+00, 9.98349404e-01, 3.31664589e+00,
  2.14298956e+00, 2.26174414e-01, 9.84736340e-01,-3.41221887e-02,
  2.47034621e+00,-3.73595824e-01, 2.78915871e+00, 2.92805887e+00,
  3.97176276e-03, 5.00656613e-01, 1.48999748e-01, 3.46306362e+00,
 -2.27748739e-01, 2.19780328e+00, 9.96584871e-01, 1.43174577e+00,
  2.02556748e+00, 1.59188108e+00, 6.95808544e-01, 2.13119704e+00,
  2.84452237e-02, 1.08582518e+00, 2.59230608e+00, 1.29934121e+00,
  5.05063718e-01, 4.05205672e+00, 1.72385901e+00, 3.96660032e+00,
  2.32458758e+00, 1.96017636e-01, 2.76856357e+00, 3.82004547e+00,
  4.36079046e+00, 1.63231793e+00, 2.53402982e-01, 3.20685516e+00,
  1.63008555e+00, 7.64419198e-01, 2.56192651e+00, 1.86171562e+00,
  1.87137605e+00,-9.61029030e-01, 1.69671638e+00, 6.03860492e-01,
  1.00592263e+00, 1.87981905e+00, 8.86704692e-01, 5.40989963e-01,
  1.58512881e+00, 2.23868963e+00, 2.56884018e+00, 1.90426782e+00,
  3.00883991e+00, 8.14076371e-01, 1.28708521e+00, 2.48387122e+00,
  2.18632046e+00, 3.40011790e+00, 2.22740777e+00, 2.56328714e+00,
  3.50070262e+00, 1.77804048e+00, 9.02137402e-01, 1.47702596e-01,
 -6.24641772e-01, 6.98535931e-01,-3.13683501e-01, 1.18241491e+00,
  2.58824548e+00, 2.75122452e+00, 3.19966183e+00, 4.09196946e+00,
  1.79379462e+00, 2.05654854e-01, 3.34389258e+00, 3.56366609e+00,
  1.62176659e+00, 2.89326801e+00, 2.21932238e+00, 3.47935415e+00,
  2.81102890e+00, 2.76255270e+00, 9.80853479e-01, 2.38010781e-01,
  1.45006724e+00, 9.02987045e-02, 1.70977513e+00,-1.34067248e+00,
  1.44468759e-01, 1.91615501e+00, 1.03888842e+00, 1.51513108e+00,
  3.55196861e+00, 1.89965241e+00, 2.38380677e+00, 2.40228639e+00,
  2.48494280e+00, 2.75740654e+00,-2.76457781e-01, 2.92185012e-01,
  3.50453557e+00, 1.71543371e+00, 2.54413780e-01, 1.78306479e+00,
  7.17665491e-01, 2.03877285e-01, 1.55559809e+00, 1.27975141e+00,
  1.81843326e+00, 1.07567700e+00, 2.98950797e+00, 6.78723039e-01,
  5.63978306e-01, 9.03315800e-01, 2.73683669e+00, 1.86033892e+00,
  1.36883669e+00, 9.34531227e-01,-8.73029818e-01, 1.67221810e+00,
  2.33788056e+00, 5.09579941e-01,-1.58254481e-01, 1.53656503e+00,
  2.46960611e+00, 3.01155033e+00, 2.18554965e+00, 1.58027819e+00,
  2.60326233e+00, 1.51059994e+00, 1.46157728e-01, 1.09004270e-01,
  2.95970459e+00, 2.36637372e+00, 2.83971082e+00, 9.76652252e-01,
  1.70397215e+00, 3.75677533e-01, 2.47542970e+00, 2.74174153e+00,
  4.36662785e+00, 9.82878834e-02, 2.49350601e+00,-8.87265359e-01,
  8.60364966e-01, 1.26618301e-01,-6.83415762e-02, 3.61952817e+00,
 -3.39757110e-01,-1.05584917e+00, 2.32434728e+00, 1.10792746e+00,
  3.76316412e+00, 1.58393175e+00, 3.86964322e-01,-5.26299988e-01,
  1.48518696e+00, 4.99008619e-01, 8.39295429e-01, 1.01484641e+00,
 -1.02053555e+00, 2.10777016e+00, 3.69282745e+00,-9.88649079e-02,
  2.29154840e+00, 3.95262371e+00, 5.48593205e-01, 1.31918344e+00,
 -1.43427210e+00, 5.97525554e-01, 2.96370030e+00, 2.22420530e+00,
  2.84081340e+00, 4.93132944e-01, 2.84918525e+00, 2.27147846e+00,
  2.25115371e+00, 3.18533967e+00, 2.52002396e+00, 7.99433147e-01,
  1.25410550e+00, 2.09397490e+00, 7.79648508e-01,-3.15132210e-01,
  4.09034541e-02, 4.10398000e+00, 6.85488935e-01, 1.49997020e+00])

    elif xind == 2:
        # IIHI Loss = 5.732753637478235e-14
        Xvec_nd16 = np.array([ 2.18613281, 2.09059276, 0.62441776, 2.28084261, 0.43502673, 1.95774143,
  1.10447539, 2.60852773, 2.61805363, 1.73523998, 2.01501315, 0.34097336,
  1.67867356, 0.82782399, 1.5706314 , 3.18649907,-0.13690971, 2.92908389,
  1.80300866, 2.75705644, 1.86359587, 3.30406198, 1.232304  , 0.00771825,
  1.40540918, 2.20908075, 0.95563347, 0.26095726, 1.64330022, 0.11887155,
  1.56441081, 0.96000211,-0.16642962, 1.4517079 , 4.23663133, 0.9194044 ,
  0.32429818, 1.55004573, 2.80454244, 0.19730858, 0.87400439, 2.30235771,
  3.89889631, 3.69397664, 2.80710184, 1.89131874, 0.05230204, 1.56795541,
  3.83147023, 2.84096888,-0.38455186, 2.05705198, 0.40551546, 1.10415553,
  1.34798504, 1.66319713, 2.89535602, 2.35676146, 2.12844192, 0.62588102,
  2.48571662, 0.00489202, 1.00530904, 0.74415582, 1.29177143, 1.55742625,
  1.94087932, 2.06150761, 0.58166293,-0.56870209, 2.97043916, 0.96784766,
 -0.71655351, 0.30797682, 1.78846344, 1.81567999, 3.73332877, 2.0349914 ,
  2.89319999, 2.05763098, 3.21589042, 0.99270506, 3.03743365,-0.0272026 ,
  1.76359362, 0.39395449, 0.49991153, 1.0544266 , 0.73295081, 1.92726185,
 -0.56511039, 3.92894133, 1.43043189, 1.02940824, 1.08535872, 0.28825851,
  1.92411676, 1.37420124, 2.47379315, 1.53002044, 1.25803999, 0.47248378,
  1.89149897, 1.66662458, 1.52642032, 2.64614853, 0.74273548, 2.17276908,
  0.24384131, 0.88908919, 2.73182207, 3.45984705, 3.37342954, 0.95428594,
  1.05274509, 0.84790559, 1.70368139, 0.24173035, 1.59729434, 2.20287574,
  1.2471386 , 2.75408259, 2.6201593 , 2.67505606, 1.36468364, 1.0158321 ,
  2.59291023, 1.27409163, 1.99592704, 2.50449239,-0.39824538, 1.7607615 ,
  1.65567381, 1.1730929 , 2.26203005, 1.79629526, 0.17171615, 0.70632753,
  0.70313189, 1.93338053, 3.12250318, 1.68121225, 2.29302667, 2.28482373,
  3.17341384, 0.73881747, 0.81249387, 1.63250641, 1.44859504, 0.08282525,
  0.22680986, 2.12267568, 2.15746286, 1.47197811, 0.86860667,-0.23645686,
  2.1169286 , 1.47429905, 1.69010854, 0.43292789, 0.19876498, 1.49789849,
  1.8308929 , 1.71040233, 3.10590743, 0.73774266, 3.56998865, 1.49495883,
  0.73320788, 0.05518849, 1.87480644, 0.8359156 , 1.35548746,-0.7479531 ,
  3.58791836, 1.50940064, 0.9237668 , 2.93402068, 0.99545289, 1.26578011,
  1.65390546, 1.63860885, 0.7300503 , 2.5186912 , 3.30751624, 2.71134661,
  0.25377023,-0.11515395, 2.10162382, 0.53632847, 3.2262329 , 0.60148212,
  1.69082008, 0.84186649, 0.99808819, 0.91269314, 2.01461696, 1.87580707,
 -0.19625747, 1.49194398, 2.78773303, 2.74376861, 0.47983116, 1.08508974,
  0.70145855, 0.35958557, 1.59887817,-0.54108485, 1.65516902, 3.94339707,
  1.44840279, 1.33207827, 0.98717452, 2.3804742 , 1.89446397, 0.26616261,
  0.68769542, 0.38853291, 1.82408864, 1.12137999, 2.07846176, 2.06175804,
  0.26337837, 2.74490376,-0.027246  , 0.5093753 , 1.68248817, 2.62817602,
  1.20274751, 1.19197184, 1.08286709, 0.70515992, 1.83391356, 0.99772434,
  1.37729385, 0.06130231, 0.84220912, 0.93063955, 0.11708941, 1.52951402,
  2.30754128, 0.9295235 , 0.18400496, 0.79846871, 1.14666947,-0.11664611,
  1.92864669, 2.06029154, 0.21171041, 1.03875762, 1.25988713, 0.39121678,
  1.44690738, 1.29994932, 3.01229079, 2.65749765, 1.6769613 , 0.42268307,
  2.72121415, 0.90077626, 2.86990058, 2.21707795, 0.86754731, 2.13354482,
  0.80949528, 0.56205714, 1.06904188, 2.09327672, 0.56712135, 1.90001709,
  2.99125112, 1.12832582])

    elif xind == 3:
        # IIIH Loss = 1.3261232110914857e-13
        Xvec_nd16 = np.array([ 1.79655224, 2.48795848, 1.41551496, 1.04426581, 2.56617374, 2.33391784,
  2.72920023, 2.6858474 , 2.23765135, 0.76460023, 0.16252336, 2.84068079,
  1.89098768, 2.42877458, 2.1012257 , 0.48163799, 1.8553364 , 4.17108584,
 -0.01589209, 1.9037535 ,-0.267744  , 0.31590803, 2.1709137 , 1.2288222 ,
  2.18227822, 0.61951604, 1.78228537, 3.28497053, 1.15007032, 1.35512072,
  2.35618587, 1.56854954, 3.29052119, 0.46685803, 2.07104146, 2.35479788,
  1.70980281, 1.20719221, 2.21069733, 2.47045226, 0.36010975, 1.80723156,
 -0.19531448, 0.69052683, 1.69583552, 1.16992414, 3.26598407, 1.94410185,
  1.24195116,-0.22211397, 1.23341329, 2.87030547, 1.42234867, 3.09928458,
  1.0423891 , 2.69392901, 0.50800505, 1.71578296, 2.35412599, 0.82169146,
  2.24173522,-0.47609495, 0.13320572, 3.81016687, 2.79261351, 0.15050932,
  1.61751361, 1.49732419, 0.87122826, 2.10161777, 0.06085214, 0.48120171,
  2.44131171, 2.29828781, 0.98578942, 0.70019104, 3.49757399, 1.19515344,
  0.57395581, 0.12899923, 1.88648647,-0.49892097, 1.88266656, 2.29221094,
 -0.59367834, 1.57745718, 0.42659893, 1.44813094, 2.67319406, 1.28196955,
  1.30015368, 3.06272174, 1.06805086, 1.6319883 , 3.36483383, 3.50109413,
  1.48468158, 2.37541078, 2.00148879, 0.81828136, 2.72107149, 1.87216975,
  0.48570844, 1.18265134,-0.95970227, 0.84349071, 3.35021683, 1.48817602,
  3.11343056, 1.33363601, 2.74494207, 2.99914351, 2.83713999, 0.30273846,
  0.29579415, 0.71461042, 2.23496703, 3.78400803, 0.86131877, 1.83194927,
  1.53848676, 0.00740318, 1.64202723, 3.40335456, 1.8066818 , 0.26643739,
  3.51707512, 1.7261525 , 0.51413737, 1.18853283, 0.14249003, 2.2310168 ,
  1.30072792, 1.76763803, 1.03920977, 1.89028911, 1.60991078, 1.5109697 ,
 -0.65409065, 1.98291282, 2.46189208, 0.12796808, 1.32323625, 1.79986428,
 -0.34235984, 0.98404133, 1.83841194, 2.02775114, 1.17462539, 0.74069677,
  3.48319325, 0.77052511, 2.27098029, 1.04784261, 0.0730024 , 1.37943549,
  2.79185701, 1.87932905, 2.13670085, 2.56242504, 2.71839486, 2.21100478,
  1.24302861, 3.1422542 , 2.2830637 , 0.89676728,-0.11011174, 1.6558912 ,
  1.25331256, 1.13788933, 1.95566702, 0.08067508, 2.61142559, 0.24275338,
  3.21500694, 0.30138187, 1.48115826, 1.18495021, 0.28282137, 1.16921482,
  2.04298895, 0.21892933, 3.05119214, 1.849248  , 3.41088981, 2.08263253,
  2.73948224, 0.91673854, 0.90486484, 2.18902769, 0.18226907, 0.42038729,
  1.86234575, 1.43477476, 2.672149  , 1.47480963, 3.29107782, 1.22156006,
  1.23605339, 2.81087187, 0.42493469,-0.4393887 , 1.38215292, 3.23854901,
  2.13333142,-0.61266431, 1.56743011,-0.38394749, 1.43128398, 2.53052628,
  0.58301424, 3.68660021, 1.27138165, 0.31669092, 2.33946005, 1.32907737,
  0.0379009 , 1.7411734 , 0.76810712, 3.59779283, 1.93590451, 0.94108111,
  0.69245321, 1.51951651, 0.86637409, 1.20063542, 2.26178173, 0.66719032,
  2.85145722, 1.97326672, 0.71257113, 0.59000692, 0.77319814, 2.36086644,
  2.13765333, 2.65856898, 1.52634673, 0.56223525, 1.01679941, 2.24885792,
 -0.02582182, 3.12527956, 1.29954517, 4.07020949, 0.4784771 , 2.17705996,
  2.62583661, 1.10381708, 2.04000178, 0.23552374, 1.57307816, 1.62191792,
  1.57628092, 1.04985352, 2.28689733, 0.89905987, 2.23108576, 1.01155412,
 -0.50530795, 2.30857885, 1.86831614, 1.12881445, 3.60218192, 1.9190149 ,
  1.56482272, 2.00537422, 0.7605822 , 3.28673242, 0.17177288,-0.42451912,
  1.57779969, 1.73760724])

    else: # xind == 0
        # HIII Loss = 3.78998574575783e-14
        Xvec_nd16 = np.array([ 0.75693028, 0.1438011 , 2.11857507, 1.26990427, 1.47885035, 1.61498347,
  1.27163499, 1.76598709, 0.86366121, 1.04901   , 0.22101929, 2.43601452,
  1.57284709, 1.11231272, 2.76445802, 0.97765873, 1.35114733, 2.19088539,
  2.84943284, 3.86713757, 5.04952692, 3.64222876, 0.72498763, 0.85381571,
  0.05136186, 0.37725966, 1.54360786, 2.2101346 ,-0.70993815, 3.2296124 ,
 -0.26690793, 2.87828452, 2.51512368, 1.52235824, 2.14746054,-1.56433771,
  0.20866252,-0.32243481, 3.21661982, 1.74091349, 3.46062874, 0.89574062,
  1.94171186, 2.0969479 , 2.92706201, 3.26754816, 1.33247229, 1.99793915,
  2.59004548, 0.6148506 , 0.37150111, 0.78663617, 1.96153941, 1.60346846,
  2.54882183, 2.57691316, 2.23485598, 3.59098245, 2.16923566, 3.00814927,
  1.25080046, 2.79246355,-0.30295015, 2.69513716, 4.53324774, 2.18906234,
  2.040591  , 0.96712319, 3.32664926, 2.88544012,-0.80419447,-0.30157179,
  1.06222178, 1.74030778, 0.38013084, 1.27994141, 1.5968077 , 2.21222686,
  2.27207053, 0.22075187,-0.23308857, 1.20783309, 3.16666548, 1.039655  ,
  2.03851861,-0.31520438, 0.40923714, 3.74688789, 3.10899344, 0.93886833,
  3.45133617, 1.47804048, 0.5330267 , 1.5491153 , 3.70980124, 3.06638579,
  3.3821775 , 1.21446365, 1.44113532, 3.27246513, 1.84888671,-0.86317478,
  0.44504415, 1.09977853,-1.03843952, 3.56968195, 0.25906993, 1.30213568,
  0.67691612, 2.82388759, 1.8414816 , 1.13482648, 4.34930992, 2.3752475 ,
  0.83433566, 2.92179194, 2.63793247, 3.32402662, 3.66012296, 0.43775914,
  0.38970348, 0.94297737, 1.51056625, 2.36213513, 0.24818239, 2.03895396,
  2.59195133, 0.2012857 , 0.59932994, 4.25536643,-0.04272501, 0.99175125,
  0.96726907, 3.03159263, 3.14005395, 2.11904306, 2.91031271,-1.12180801,
  2.62912662, 1.65593867,-0.13609016, 2.45568616, 2.21264263, 0.6098842 ,
  0.40810536, 1.02032634,-0.74332438, 2.00927731, 1.62395274, 1.92576334,
  1.9658005 , 3.11352727, 1.52069689, 0.15566056, 3.31487566, 4.42369788,
  4.21613956, 2.34793698, 3.61875212,-0.45584795, 3.60478569, 2.68885419,
  3.76146077, 3.6691208 , 2.48750973, 0.73491873, 0.35154738,-0.9628904 ,
  0.23453844, 3.24183259, 2.81930779, 1.6040768 , 2.82985957, 2.42841304,
 -0.14890251, 3.97620343, 0.8288304 , 0.24874912, 1.58204359, 1.89790363,
 -0.08156693, 1.89608442, 3.56778258, 3.91805594, 1.0258022 , 2.29997413,
  2.63972327, 0.35911631, 3.87634476, 2.2444382 , 0.52030309, 0.38845916,
  1.80766952, 1.54540891, 2.07556723, 0.92308529,-0.10553571, 0.32347583,
  3.73071548, 0.81943079, 2.7012096 ,-0.32094206, 0.70549198, 2.8541014 ,
 -0.23035286, 1.08404871, 2.53602947, 1.4475729 , 0.9120777 , 2.58325045,
  3.31349451, 0.78894955, 2.44722058, 1.4891641 , 1.60995246, 2.52800259,
  1.14816455,-0.78158384,-0.73376609, 0.71020854, 1.18574638,-0.32433616,
  2.02631268, 0.54565522, 0.93740527, 2.34686128, 0.57083031, 1.58370023,
  1.4467342 , 0.34231912, 1.32906186, 1.6150305 , 1.26724663, 0.07048724,
  2.31401657,-0.07629447, 0.55883028, 1.97960422, 0.42084408, 2.66553358,
  3.35065138, 1.52717509, 2.12913503, 0.039693  , 4.02118702, 2.86559296,
  2.43680002, 0.7765592 , 2.90034908, 0.16388971, 1.1895093 ,-0.74486609,
  1.90088913, 2.30004136, 1.60668301, 1.87205154, 0.69379922, 0.43660155,
  0.80998181, 1.35637279, 1.0973559 , 0.56320101, 0.63389352, 0.78989662,
  0.67528838, 1.40389125, 2.24079184, 2.03809061, 1.08048627, 2.03847497,
  2.26808798, 1.06582639])

    return snap_disp_ansatz(Xvec_nd16, nfock)

In [ ]:
def get_yr_snap_four_qubits(xind):
    """
    SNAP-displacement circuit for W[j] with j = 0-3.
    """
    # Initialize
    nfock = 16

    # Parameters
    if xind == 1:
        # IWII Loss = 1.790371055585707e-14
        Xvec_nd16 = np.array([ 7.53967510e-01, 2.23544904e+00, 1.90102982e+00, 1.21603454e+00,
  1.74652009e+00, 1.32938563e+00, 1.09483774e+00, 1.19498588e+00,
 -1.54421759e-01, 1.97727245e+00, 2.08299170e+00, 1.90653157e+00,
  1.51314796e+00, 2.49307049e+00, 1.71236206e+00, 1.69633005e+00,
  3.37563666e+00, 1.89060298e+00, 8.48705498e-01, 2.50979954e+00,
  2.24245436e+00, 2.96847322e+00,-4.89133268e-01, 3.14844545e+00,
  2.68905928e+00, 4.39690746e-01, 2.14925771e+00, 1.57228616e+00,
  2.56003431e+00, 1.84059021e+00, 2.67724926e-01,-1.33015780e-01,
  2.60539691e+00, 2.69706736e+00, 1.57181039e+00, 2.62104071e+00,
  1.06214319e+00, 1.67836510e+00, 1.77563072e+00, 2.68158831e+00,
  3.26283292e+00, 4.80569858e-01, 1.45064568e+00, 1.98845311e+00,
  2.88000951e+00, 1.21766629e+00, 2.33053708e+00, 2.83438986e+00,
  1.17836886e+00, 3.28065737e+00, 2.61857758e+00, 4.14803068e-01,
  2.16237197e+00, 2.91144772e+00, 4.34358970e-01, 9.75911715e-01,
  2.88186625e+00,-1.72469967e-01, 4.40503382e+00, 2.30656138e+00,
 -1.73157866e-02, 1.42587391e-01, 6.47071160e-01, 2.18007741e+00,
  1.05853687e+00, 2.67616663e+00, 1.38441148e+00, 3.10224022e+00,
  2.03215601e+00, 1.44902719e+00, 4.00322520e+00, 3.75290344e+00,
  2.95028078e-02, 1.61355554e+00, 3.47884550e-01, 1.11019388e+00,
  2.77381541e-02, 1.84566220e+00,-3.48920882e-01, 7.73518843e-01,
  1.04869674e+00, 9.23188072e-01, 9.12211777e-01, 2.41612360e+00,
  2.42471740e+00,-3.35784185e-02, 1.23448523e+00, 2.69604831e+00,
  1.40681492e+00, 1.84167059e+00, 1.34416725e+00, 2.06287974e+00,
  1.89244086e+00, 1.12988070e+00, 1.75667464e+00, 1.92755911e-01,
 -6.21313477e-01, 2.08782317e+00, 3.81539846e+00, 1.69138503e+00,
  6.05497259e-01, 9.12584635e-01, 2.50416839e+00, 5.64440695e-01,
  1.88381578e+00,-3.14259885e-04, 2.66719020e+00, 1.56601325e+00,
  1.69755619e+00,-1.75326904e-01, 1.12224381e+00, 1.13755381e+00,
  1.72966285e+00, 1.49908211e+00,-1.13143152e-01, 2.60847586e+00,
 -1.15713027e+00, 1.17359095e+00, 2.73064115e+00, 1.52598850e+00,
  1.20363996e+00, 2.90195030e+00,-5.25178483e-01, 4.01665263e+00,
  6.50387035e-01, 1.62932709e+00, 2.57332616e+00, 2.58573183e+00,
  2.17429556e+00, 1.06400596e+00, 2.16871253e+00, 2.32566791e+00,
  3.59619794e+00, 5.80003208e-01, 1.02097227e+00, 7.81774760e-02,
  2.95639960e+00, 3.16976408e-02, 6.36799765e-01,-1.80744169e-01,
  1.53664777e-01, 1.37337297e+00, 2.69911160e+00, 2.56402827e+00,
 -3.87155101e-01, 2.17129297e+00, 9.51706584e-01, 2.83893635e+00,
  3.39302688e-01, 4.91847538e-01,-7.36729158e-01, 2.31901327e+00,
  1.78361342e+00, 3.75407654e+00, 7.51709179e-01, 2.31616764e+00,
  1.56671609e+00, 1.02347857e+00, 1.30796369e+00,-4.34819666e-01,
  2.22262798e-01, 6.85050485e-01, 2.19899395e+00, 3.09388014e+00,
  1.41747175e+00, 1.44913521e+00, 2.79167779e+00,-5.78888485e-01,
 -2.08899909e-01, 2.88859173e+00, 2.39581399e+00, 1.49127258e+00,
  1.53619267e+00, 2.21007523e+00, 2.14280576e+00, 1.51395174e+00,
  2.22907168e+00, 3.09281831e+00, 3.25538954e+00, 4.54746317e-01,
  1.52007644e+00, 2.61281183e+00, 1.00710304e+00, 2.16345799e+00,
  3.25917665e+00, 1.25307904e-01, 2.01415874e+00, 1.94796220e+00,
  4.91054715e-01, 1.26827781e+00, 2.91046277e+00, 1.85243403e+00,
  8.78844798e-01, 3.71610060e-01, 1.61418743e+00,-4.04062463e-01,
  3.20255249e+00, 1.05141089e+00, 4.59579833e-01, 4.50114721e-01,
  2.36185824e+00, 3.68483310e-01, 1.42043279e+00, 5.94985696e-01,
  1.37109032e+00, 3.46226997e-01, 2.79255980e+00, 2.55107952e+00,
  2.87244726e+00,-5.01982027e-01, 1.77685232e+00, 3.27499035e+00,
  1.62557602e+00, 1.75144150e+00, 2.65694176e+00,-1.29609450e-01,
  3.24869182e+00,-3.71158767e-01, 8.10358631e-01, 1.25217964e-01,
  2.97484718e-01, 1.18262170e+00, 1.26853785e+00, 8.23628481e-01,
  1.33620506e+00, 3.46900097e-01, 8.69900938e-01, 1.20829089e-01,
  2.91401132e+00, 3.34475396e+00, 3.61402670e+00, 1.58566495e+00,
  2.22900181e+00, 1.98106388e+00, 1.72808103e+00, 2.61468030e+00,
  2.42824291e+00,-8.17082039e-01, 1.49131117e+00,-2.17107317e-01,
  2.17514345e+00, 1.13784026e+00, 3.05601884e+00, 5.72597320e-01,
  2.71914782e+00, 6.03159841e-01, 1.40871725e+00, 2.91167878e+00,
  1.24936595e+00, 2.45265531e+00, 3.38664926e+00,-7.70174038e-01,
  3.98408926e-01, 2.11124938e+00, 2.62799617e+00, 1.23377490e+00,
  6.84943800e-01,-7.85967123e-01, 1.28460304e+00, 2.09711636e+00,
  5.00523705e-01, 2.36771771e+00, 1.16073875e+00, 3.45885954e+00,
  1.86740558e+00, 2.68276534e+00, 1.62495705e+00, 3.01412515e+00,
  2.33456174e+00, 2.53190334e+00, 2.68594131e+00, 9.05407181e-01])

    elif xind == 2:
        # IIWI Loss = 2.1512104564148039e-13
        Xvec_nd16 = np.array([ 2.14287758, 0.49287997, 1.89620921, 0.30956438, 2.65936594, 1.50952092,
  0.48945441, 1.94193043, 0.71663636, 1.96512253, 1.54016746, 2.59178421,
  0.95075349, 2.65737001, 1.60349364, 0.61622231, 0.14628279, 1.8630508 ,
  2.72337121, 2.21608217, 3.73319631, 0.58260157, 1.80975849, 2.67585397,
  0.45207189,-0.61088465, 2.0324895 , 1.67514167, 1.5465487 , 1.62035001,
 -0.35563163, 2.71556456, 2.18987155, 2.00905649, 0.42966085, 3.55881035,
  1.03958665, 0.35118405, 2.29073315, 1.37730439, 1.25767704, 2.06382918,
 -1.36896692,-0.43407051, 3.68682397, 1.16843961, 2.41811776, 1.89476363,
  0.70938981, 2.21602106, 0.84775949, 1.07135203, 1.56195959, 3.09120921,
  1.29861124, 0.7110958 , 0.54747287, 2.0493727 , 3.68991999, 0.244465  ,
  2.36791248,-1.67026597, 0.68433414, 1.71121581, 0.6215698 ,-0.29271276,
  2.2820471 ,-1.45538216, 3.5304771 , 2.71635613, 0.36894749, 3.30420485,
  1.26115366, 1.72125028, 0.74975731, 0.79909751, 0.76742474, 3.80225533,
  2.20624354, 1.91295257, 2.77587479, 2.85238775, 0.0887186 , 0.60380355,
  2.44170994,-0.3697785 , 2.78455698, 0.59390451, 1.05977414, 2.0092477 ,
  2.26194903, 2.9657397 , 2.86863715, 3.0478515 , 1.16122364, 1.49882114,
  2.69763769, 2.55571277, 1.35254885, 1.22756756, 1.66874759, 1.74857096,
  0.97251008, 1.75316855,-0.36855485, 1.12294617, 0.34188782, 0.02895356,
  2.38804942, 2.11218607, 1.48487956, 4.51842965, 2.58644298, 1.59521482,
  1.38091965, 2.5544672 , 0.82331711, 0.69974927, 1.08809855,-0.51761195,
  3.45539839, 3.21581309, 0.99953007, 2.51793351, 0.66340406, 2.31055525,
  3.00080975, 1.14708864, 0.79944694, 1.04448413, 2.08447245, 0.92178884,
  2.005903  , 0.96868397, 3.17033419, 3.80076031,-0.18668121, 4.35149243,
 -0.21362974, 1.8286222 , 0.60070629, 1.90647174, 2.36544621, 2.30719914,
  1.56571779, 2.32658603, 1.95231034, 3.48647152,-0.06099977, 1.33236061,
  3.96840753, 2.05098127, 0.41845316, 0.49162971, 2.52709581, 1.77058345,
  2.45357018,-0.07403417, 1.51901097, 2.87327458, 1.99935888,-0.33635357,
  0.16400178, 2.72352195, 3.33141076,-0.61088115, 1.5529886 , 4.31009979,
  0.80667114, 0.48921144, 0.68369536, 2.551378  ,-0.49867287, 0.22859215,
  2.6909177 , 1.27613952, 0.69427844, 1.66172726, 1.22736376, 2.16906417,
  2.70668623, 2.16727223, 2.06299285, 3.00831446, 2.96610958, 1.57994437,
  1.77953186, 1.07426322, 0.09495905, 1.03921554, 0.71153098, 1.76318632,
  2.69525853, 1.16301508, 1.12368799, 1.76909241, 0.06621843, 1.05499909,
 -0.27404833, 2.20577187, 0.55646691, 1.18572313, 1.0116285 , 3.86454391,
  1.2089564 , 0.45189911,-0.05379134, 0.73159337, 0.89895693, 0.37211182,
  2.38102373, 3.40862555, 1.12144575,-0.79178535, 2.71179405, 3.09705349,
  1.12193809, 0.76327383, 0.71269362, 1.33772718, 2.16729888, 3.01310918,
  2.86694445, 1.52731191, 1.02706432, 2.81552457, 1.38991825, 2.81253807,
  2.80282409, 2.7346001 , 0.7416932 ,-0.05286587, 0.91430552, 1.0615201 ,
  1.9638997 , 1.3353072 , 1.8753455 , 2.04949756,-0.48197358,-0.58739947,
  2.99227799, 3.79310638, 1.07148103, 2.98184619, 2.79167385, 2.44615823,
  1.86076999,-1.05422518, 1.75633447, 1.52351208, 0.84147532, 0.79944759,
 -0.00724691, 2.78311336, 1.39786202, 1.02585769, 0.99490739, 1.07419232,
  0.72191917, 2.84677079, 1.7625317 , 2.12599546, 2.273095  , 0.27304724,
  2.325824  , 3.4861069 , 1.67860867, 2.57557535, 3.18552571, 3.85723142,
  2.09819284, 1.86936982])

    elif xind == 3:
        # IIIW Loss = 5.2078860897763425e-14
        Xvec_nd16 = np.array([ 3.67603703e-01, 2.83266470e+00,-1.07893320e-01, 3.36339971e-01,
  1.67573836e+00, 1.45369108e+00, 4.19461525e-01, 2.65872571e+00,
  2.44054092e+00, 2.66214882e+00, 1.60798999e+00, 5.25208956e-01,
  1.16286329e+00, 2.36636568e+00, 1.68252475e+00, 1.95051200e+00,
  1.01443331e+00, 1.47588443e+00, 2.00266972e+00, 1.86859356e+00,
 -2.53525571e-01, 2.30657110e+00, 1.44803048e+00, 1.55956926e+00,
 -1.39375120e-01, 2.28596113e+00, 2.32502717e+00, 2.63999173e+00,
  3.36932971e+00, 3.35870494e+00, 1.35904332e+00, 1.30566225e+00,
  1.34802956e+00, 2.64454085e+00, 2.00418811e+00, 1.15961445e+00,
 -6.27427688e-01, 3.30370218e+00, 1.18380120e+00, 7.05068722e-01,
  1.87938411e+00, 4.65586442e-01, 1.95567825e+00, 2.41472762e+00,
  1.00266648e+00, 1.22104854e+00, 2.23419941e+00, 2.43452933e+00,
  5.47987578e-01,-1.15823865e+00, 3.82065876e-01, 2.20423702e+00,
  4.72002895e+00, 4.05807444e-02, 5.02576891e-01, 3.32635160e+00,
  1.70404982e+00, 1.80960029e+00, 7.18165478e-01, 2.78335355e+00,
  4.27770472e+00, 3.05612769e-01, 2.45972265e+00, 2.82674398e+00,
  4.21708972e+00, 1.79748493e+00, 3.11134142e+00, 1.54733187e+00,
  1.79544021e-01, 1.75554130e+00, 1.67282814e+00, 1.51544878e+00,
  9.03110508e-01, 1.96849194e+00, 2.29787547e+00,-1.50234414e-03,
 -8.32049239e-01, 2.31196916e+00, 3.13107145e-01,-1.22462874e+00,
  9.60586068e-01, 9.84799764e-01,-5.88682667e-02, 6.10234899e-01,
  5.62869152e-01, 2.93296677e+00,-5.90681433e-01, 3.55403736e+00,
  1.76817098e+00, 1.02804304e+00, 1.48088569e+00, 1.66616585e+00,
  1.72389830e+00, 9.58653561e-01, 2.93240091e+00, 1.35901492e+00,
  6.99216492e-01, 1.75705076e+00, 8.01294613e-01, 9.73057200e-01,
  1.40030070e+00, 3.99956765e+00, 1.87817475e+00,-2.10127874e-01,
 -1.65717964e-01, 3.06005405e+00, 2.72420142e+00, 2.08750087e+00,
  5.93787984e-02, 4.34785456e+00, 2.30198319e-01, 3.15011560e+00,
  1.80747303e+00, 2.69718463e+00, 4.00082146e+00,-4.19589785e-01,
  9.97268827e-01, 8.84411808e-01, 2.58900143e+00, 3.55286980e+00,
  3.34694490e+00, 8.13636050e-01, 1.00680170e+00, 5.82959610e-01,
  3.15755380e+00, 1.56393113e+00, 9.46544392e-01, 1.45868930e+00,
  1.54571853e+00, 2.18646750e-01, 8.02068023e-01, 1.53493837e+00,
 -4.87183059e-01, 1.94556258e-01, 3.38030962e+00, 2.25844843e+00,
  4.02166915e-01, 9.33986264e-02, 3.70310395e+00, 2.11254392e+00,
  1.44698155e+00, 1.60448477e-01, 1.55422666e+00, 7.56258450e-01,
  4.24743177e-01, 2.69580521e+00, 1.38181531e+00, 6.33261999e-02,
  8.94747119e-01, 5.36036532e-01, 2.20335724e+00, 3.18308973e+00,
  3.52125866e+00,-8.16590150e-02, 5.05109586e-02, 1.78877783e+00,
 -7.26965448e-02, 2.46295207e+00,-8.32207316e-01, 9.31102800e-01,
  3.98715121e+00, 1.34605433e+00, 1.61924715e+00, 1.77533637e+00,
  3.52395826e+00, 1.34793304e+00, 5.32625783e-01, 1.71405199e+00,
  8.25452384e-01, 3.44340506e+00, 3.20568042e+00, 3.86195966e+00,
  9.02732277e-01, 1.29178085e+00, 3.02154128e+00,-6.81759637e-01,
  4.45566743e-01,-3.11584942e-01, 7.67759960e-02,-4.77235037e-01,
  1.94737395e+00, 2.86177824e+00, 2.72492760e+00, 3.01731756e+00,
  2.74957305e+00, 2.83734819e+00, 3.86407257e+00, 1.63895317e+00,
  3.76108140e-02,-1.28781150e-01, 1.80270426e+00, 2.60802158e+00,
  4.15286104e+00, 3.33167214e+00, 2.03687380e+00, 3.99926216e+00,
  8.30438551e-01, 3.68419924e-01,-8.96183779e-01,-3.72660729e-01,
  9.62013802e-01, 2.62298401e+00, 7.84463938e-01,-4.13704414e-01,
  2.46590161e+00, 5.09801117e-01, 1.26642622e+00,-4.47021492e-01,
  5.58400380e-01,-3.08303762e-01, 3.83809808e-01, 2.18794426e+00,
  2.18530704e+00, 3.22551476e+00, 2.61822495e+00, 1.61595992e+00,
  1.49015884e+00, 1.11753620e+00, 2.06119726e+00, 2.04862263e-01,
 -5.84482441e-02, 2.76218292e+00, 3.40385836e+00, 2.44385990e+00,
  2.60580333e+00, 1.91237870e+00, 2.81053845e+00, 1.67524335e+00,
  3.38199793e+00, 5.79008201e-01, 2.80532924e+00, 2.43899418e+00,
 -3.21162513e-01, 4.97064292e-01, 2.93597303e+00, 1.81298099e+00,
 -3.17650454e-01, 3.07357667e+00, 6.50910980e-01, 2.96874340e-01,
 -9.93765712e-03,-3.69495006e-01, 9.57527264e-01, 1.60409513e+00,
  9.35995784e-01, 2.80113671e+00, 2.99214035e+00, 3.30172623e+00,
  2.83596713e+00, 1.53403855e+00, 1.40341388e+00, 1.18752336e+00,
 -5.45700181e-01, 1.34524637e+00, 3.21139697e+00,-3.88050432e-01,
  3.81851578e+00, 2.15042268e+00, 1.25692205e+00, 1.08192822e+00,
  8.11055531e-01, 2.07743620e+00, 3.15876040e+00, 3.35432677e+00,
  1.48985938e+00, 2.84071797e+00, 3.67564503e-03, 1.80150185e+00,
  3.07340009e+00, 7.51296204e-01, 2.98995422e-01, 1.29614705e+00])

    else: # xind == 0
        # WIII Loss = 1.8492914808745887e-14
        Xvec_nd16 = np.array([ 2.76564163, 2.28140324, 1.98871428, 1.58417582, 2.61789081, 2.49769663,
  2.50429314, 0.29047528, 1.70891673, 0.29789922, 0.39030857, 1.36992163,
  0.49669341,-0.07115917, 1.1188049 , 1.73539623, 1.78036915,-0.01637284,
  2.23722506, 1.33186618,-0.23986923, 1.84229899, 2.11629506, 0.35768334,
  2.24311472, 4.1034731 , 0.44197437, 0.97238112, 1.62606111,-0.28615064,
  3.23908792, 2.0785331 , 0.31382139, 0.5504825 , 1.44653626, 1.6459473 ,
  3.52685089, 1.91803688, 4.00364988,-0.3987431 , 2.41831283, 2.20877178,
  2.58543851, 2.97547003, 2.69489   , 1.3415057 , 2.29276739, 2.52891241,
  2.32553229, 0.51986494, 1.07531173, 0.36557231, 3.68676673, 0.07766602,
  3.45792403, 0.6045566 , 0.47401755,-0.09332597, 0.20644992, 2.01118395,
  0.99209819, 1.83150232, 1.31407029, 1.9542366 , 0.98226535, 0.0896546 ,
  2.15519072, 0.60486455,-0.21074661, 2.10698867, 1.98075733, 2.12252161,
  3.47620799, 1.04907037, 0.64643621, 3.89487511,-0.25246072, 1.54702459,
  1.93676166, 0.46773603, 0.36826512, 1.68956997, 1.01781875,-0.38523746,
  4.41937421, 2.92806858, 4.05133076,-0.16074538, 0.77378963, 2.16941013,
  1.23799778, 0.55089923, 0.72266354, 2.94446791, 1.46371721, 1.86654409,
 -0.30926393,-0.6023001 , 2.27531861, 1.94423478, 3.19762949, 1.41481527,
  3.12050334, 1.37896879, 0.84453433, 2.36815549, 1.48550693, 2.16467257,
  1.22486572, 3.80068172, 1.17880863, 2.22864366, 0.95349049,-0.10939496,
  0.22438436, 1.98870748, 1.40581894, 2.79951835, 1.50947981, 2.05021351,
  2.54431958, 1.24015242, 0.54479058, 3.46616231, 0.24250974, 2.95862284,
  2.28084236, 1.26074101, 1.78575088, 0.59776644, 1.88638276, 1.35113928,
  0.50483715, 1.99209213, 1.66297217, 2.58600818, 2.22727088, 0.8335725 ,
  1.56988768, 0.24271324, 2.79815397, 2.00664342, 3.24714868, 0.44004886,
 -0.85760037,-0.53469377,-1.07148788,-1.09936715, 0.11134844, 2.67214464,
  3.38625113, 3.45476117, 1.3425696 , 3.98745539, 2.40756455, 3.72963945,
  0.53943041, 0.11172532, 2.36341325, 1.26403855, 2.89520189, 4.14677313,
  1.45414934, 2.26165946, 3.62424633, 3.16830496, 1.73459469, 0.68800856,
 -0.49791787, 0.07404886, 0.90608388, 0.5031362 , 0.45792436, 2.87243342,
 -0.31584043, 0.5714099 , 4.73136681, 0.43441021, 2.3672667 , 1.89873046,
  1.6169817 , 3.05049408, 1.81833465, 2.38079207, 0.69414735, 1.54412674,
  1.87879455, 0.77396122,-1.91150336, 1.10373804,-0.15928634, 2.57259855,
  1.85511858, 0.67989166,-0.73990913, 2.74704761, 2.07138631, 1.77872729,
  2.45448223, 2.81472077, 2.08884797, 1.52298308, 0.59336622, 1.19640062,
  0.24671184, 0.90057928, 1.05341145, 2.41388035, 1.24674656, 5.24446936,
  3.64699007, 0.97968441, 1.23967078, 2.99141633,-1.23470085, 3.84070592,
  2.59003064,-0.36608698,-1.16344618,-2.07377309,-0.81347048, 3.07263944,
  1.13265485, 1.67477221, 2.56074035,-1.79281424,-1.2841421 , 0.25509525,
  0.65001543,-1.15346468, 4.02070814, 2.34451991, 3.45848162, 4.01661187,
  2.40727937, 2.39300509, 5.04776058, 2.83839054, 2.50643001, 2.18004488,
  2.92692168, 2.71724344, 1.43652994,-0.14137852, 0.07965692, 0.15816299,
  0.36727717, 2.60940931, 2.45220959, 0.18587786, 2.09244701, 1.10108181,
  1.71618904, 0.106322  ,-0.02866963, 1.93585459, 0.65453622, 1.50414017,
  0.55885064, 2.30830552, 0.41285471, 3.12417622, 1.01376046, 0.68148794,
  1.53153171, 0.38873614, 1.34454196, 2.90061648, 3.65200328, 1.44753288,
  1.97756962, 1.59626855])

    return snap_disp_ansatz(Xvec_nd16, nfock)

## Expectation value

### Probabilities

In [ ]:
def qumode_probs(psi):
    """
    Probabilities.

    Argument:
    psi -- statevector in QuTip
    """
    # Initialize
    probs = []
    nfock = psi.full().shape[0]

    # Results
    for i in range(nfock):
        basis = qt.basis(nfock, i)
        ov = np.abs( psi.overlap(basis) )
        probs.append(ov**2)

    return np.array(probs)

In [ ]:
def qubit_probs(probs, nqubits):
    """
    Converts Fock state probabilities to a dictionary of bitstrings.

    Arguments:
    probs -- List of probabilities for Fock states (output of qumode_probs).
    nqubits -- number of qubits
    """
    # Check
    nfock = len(probs)
    if 2**nqubits != nfock:
        raise ValueError("Dimension mismacth in qubit_probs")

    # Create the dictionary
    bitstring_probs = {}

    for i in range(nfock):
        # Convert the index i to a bitstring of length L
        bitstring = format(i, f'0{nqubits}b')  # Format i as a binary string of length L
        bitstring_probs[bitstring] = probs[i]

    return bitstring_probs

In [ ]:
def pauli_z_exp_val(qubit_prob_dict, pauli_indices):
    """
    <Z(p1) ... Z(pN)>.

    Arguments:
    qubit_prob_dict -- probability dictionary for all basis states
    pauli_indices -- qubit indices for Pauli-Z
    """
    expectation_value = 0

    # Loop over all possible basis states and their probabilities
    for bitstring, prob in qubit_prob_dict.items():
        sign = 1
        # Apply the Pauli operators' sign depending on the state of the qubits
        for idx in pauli_indices:
            if bitstring[idx] == '1':
                sign *= -1
        # Add the weighted probability with the appropriate sign
        expectation_value += sign * prob

    return expectation_value

In [ ]:
def pauli_exp_val_four_qubits(psi, pword):
    """
    <sigma(p1) ... sigma(pN)>.

    Arguments:
    psi -- four-qubit state
    pword -- Pauli word for four qubits
    """
    # Initialize
    nqubits = 4
    psi_new = psi.copy()
    Zind = []

    # Check
    if len(pword) != nqubits:
        raise ValueError("Dimension mismacth in pauli_exp_val_four_qubits")

    # Rotate
    for i in range(nqubits):
        if pword[i] == 'X':
            U = get_xr_snap_four_qubits(i)
            psi_new = ( U * psi_new )
            Zind.append(i)
        elif pword[i] == 'Y':
            U = get_yr_snap_four_qubits(i)
            psi_new = ( U * psi_new )
            Zind.append(i)
        elif pword[i] == 'Z':
            Zind.append(i)

    # Probabilities
    probs = qumode_probs(psi_new)
    dict_probs = qubit_probs(probs, nqubits)

    # Final
    ov = pauli_z_exp_val(dict_probs, Zind)

    return ov

### Final

We want to minimize the following weighted cost function

\begin{align*}
\min_{ \vec{\alpha}^{\psi}, \bar{\theta}^{\psi} } F
&= \sum_{n = 1}^{N_s} w_j \:
E_n (\vec{\alpha}^{\psi}, \bar{\theta}^{\psi}),
\\
E_n
&= \langle \psi_n (\vec{\alpha}^{\psi}, \bar{\theta}^{\psi}) | \: H_Q \:
|\psi_n (\vec{\alpha}^{\psi}, \bar{\theta}^{\psi}) \rangle,
\\
|\psi_n \rangle
&= \mathcal{U} (\vec{\alpha}^{\psi}, \bar{\theta}^{\psi}) |n \rangle.
\end{align*}

In [ ]:
def psi_ham_psi(psi, pwords, coeffs):
    """
    <psi| H |psi> for a four-qubit Hamiltonian,
    where <psi | psi> = 1 is assumed.

    Arguments:
    psi -- qumode state in QuTip
    pwords -- list of four-qubit Pauli words
    coeffs -- Hamiltonian coefficients
    """
    en = 0.0
    for i in range(coeffs.shape[0]):
        if pwords[i] == 'I' * 4:
            en += coeffs[i]
            continue
        en += ( coeffs[i] * pauli_exp_val_four_qubits(psi, pwords[i]) )

    return en

In [ ]:
def energy_val_basis(Xvec, pwords, coeffs, nbasis, nfock):
    """
    Compute <psi (n) | H | psi (n)>, where |psi (n)> = U |n>.

    Arguments:
    Xvec -- ansatz parameters
    pwords -- list of four-qubit Pauli words
    coeffs -- Hamiltonian coefficients
    nbasis -- Fock basis state index
    nfock -- Fock cutoff for the qumode
    """
    # Unitary
    U = snap_disp_ansatz(Xvec, nfock)

    # |psi> = U |n>
    vac = qt.basis(nfock, nbasis)
    psi = U * vac

    # Energy
    en = psi_ham_psi(psi, pwords, coeffs)

    return en

In [ ]:
def get_energies(Xvec, pwords, coeffs, nstates, nfock):
    """
    Energies.

    Arguments:
    Xvec -- ansatz parameters
    pwords -- list of four-qubit Pauli words
    coeffs -- Hamiltonian coefficients
    nstates -- number of states
    nfock -- Fock cutoff for qumode
    """
    # Check
    if nstates > nfock-1:
        raise ValueError("Wrong nstates in get_energies.")

    # Energies
    envec = []
    for i in range(nstates):
        envec.append( energy_val_basis(Xvec, pwords, coeffs, i, nfock) )

    return np.array(envec)

In [ ]:
def ssvqe_cost_fun(Xvec, pwords, coeffs, wvec, nfock):
    """
    Loss function.

    Arguments:
    Xvec -- ansatz parameters
    pwords -- list of four-qubit Pauli words
    coeffs -- Hamiltonian coefficients
    wvec -- linear cost coefficients
    nfock -- Fock cutoff for qumode
    """
    # Initialize
    nstates = wvec.shape[0]

    # Energies
    envec = get_energies(Xvec, pwords, coeffs, nstates, nfock)

    # Final
    en = 0.0
    for i in range(nstates):
        en += ( wvec[i] * envec[i] )

    return en

## VQE

In [ ]:
def ssvqe_snap(pwords, coeffs, wvec, ndepth, maxiter=100,
               method='COBYLA', verb=0, threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    pwords, coeffs -- Hamiltonian operators and coefficients
    wvec -- linear cost coefficients
    ndepth -- ansatz circuit depth
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- choose additional printing
    threshold -- error tolerance
    Xvec -- optional initial guesses
    """
    # Initialize
    nfock = 16

    # Bound parameters
    alpha_min = 0.0
    alpha_max = 10.0
    theta_min = 0.0
    theta_max = np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([alpha_min, alpha_max])
    for _ in range(ndepth * nfock):
        bounds.append([theta_min, theta_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        alpha = np.random.uniform(0, 3, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        Xvec = pack_params_snap_disp(alpha, theta)

    # Loss function
    obj_fun = partial(ssvqe_cost_fun, pwords=pwords, coeffs=coeffs,
                      wvec=wvec, nfock=nfock)

    # Energy function
    nstates = wvec.shape[0]
    envec_fun = partial(get_energies, pwords=pwords, coeffs=coeffs,
                        nstates=nstates, nfock=nfock)

    # Intermediate values
    iteration_step = 0
    print_freq = 1
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {envec_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    # Final energies
    envec = envec_fun(result.x)

    return envec, result.x